In [1]:
import os
import glob
import obspy
import obspy.io.sac.sactrace
from obspy.taup import TauPyModel
from obspy.clients.iris import Client
from IPython.display import clear_output

In [2]:
client = Client()
root_dir = "/media/elemento/Element/NTU_RW/NSFE"
os.listdir(root_dir)

['.ipynb_checkpoints',
 '0_get_events.ipynb',
 '1_get_stations.ipynb',
 '2_plot_evtsta.ipynb',
 '3_get_waveform.ipynb',
 '3_get_waveform_2001.ipynb',
 'Download',
 'Download.png',
 'Events',
 'ISC_EHB_Catalog_1980-2018',
 'README.md',
 'Stations']

In [3]:
def theo_arrival(data_dir, model_name, phase):
    """Write theoretical arrival times of P waves"""
    model = TauPyModel(model=model_name)

    for ev in os.listdir(data_dir):
        print(ev)
        inpath = os.path.join(data_dir, ev)

        for sac in glob.glob(f"{inpath}/*.SAC"):
            try:
                st = obspy.read(sac)
                sachd = st[0].stats.sac
                if sachd.get("t1") != None:
                    print("Done already~")
                    continue
                print(st)

                # Set P-wave first arrival
                distaz = client.distaz(sachd["stla"], sachd["stlo"], sachd["evla"], sachd["evlo"])
                gcarc = distaz['distance']
                arrivals = model.get_travel_times(source_depth_in_km=sachd["evdp"],
                                      distance_in_degree=gcarc, phase_list=[phase])

                sachd["t1"] = arrivals[0].time + sachd["o"]
                sachd["kt1"] = arrivals[0].name

                sacf = "{}.{}.{}.{}.SAC".format(st[0].stats.network,st[0].stats.station,st[0].stats.location,st[0].stats.channel)

                st[0].write(f"{inpath}/{sacf}", format="SAC")

                clear_output(wait=False)
                
            except:
                continue

In [4]:
providers = ["IRIS", "GFZ"]
years = [2000, 2001, 2002, 2003, 2004]

for year1 in years:
    year2 = year1 + 1
    for provider in providers:
        print(f"For the range: {year1}-{year2}, and provider: {provider}")
        
        data_sac = f'{root_dir}/Download/{provider}/SAC/{year1}'
        
        # Write theoretical arrival times
        theo_arrival(data_sac, "iasp91", "ttp")
        
        clear_output(wait=False)

For the range: 2000-2001, and provider: IRIS
For the range: 2000-2001, and provider: GFZ
For the range: 2001-2002, and provider: IRIS
For the range: 2001-2002, and provider: GFZ
For the range: 2002-2003, and provider: IRIS
For the range: 2002-2003, and provider: GFZ
For the range: 2003-2004, and provider: IRIS
For the range: 2003-2004, and provider: GFZ
For the range: 2004-2005, and provider: IRIS
For the range: 2004-2005, and provider: GFZ
